In [2]:
import numpy
import pandas as pd 

In [62]:
file_path = "/Users/oliviergisiger/OneDrive - Hochschule Luzern/Eigene Dateien/DE_GD - Project/Tableau/acc_death_strnams-fixed.csv"

acc_data = pd.read_csv(file_path, sep=";")
acc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2899 entries, 0 to 2898
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   2260 non-null   float64
 1   AccidentUID                  2260 non-null   object 
 2   AccidentType                 2260 non-null   object 
 3   AccidentType_de              2260 non-null   object 
 4   AccidentSeverityCategory     2260 non-null   object 
 5   AccidentSeverityCategory_de  2260 non-null   object 
 6   AccidentInvolvingPedestrian  2260 non-null   object 
 7   AccidentInvolvingBicycle     2260 non-null   object 
 8   AccidentInvolvingMotorcycle  2260 non-null   object 
 9   RoadType                     2260 non-null   object 
 10  RoadType_de                  2260 non-null   object 
 11  AccidentLocation_CHLV95_E    2260 non-null   float64
 12  AccidentLocation_CHLV95_N    2260 non-null   float64
 13  CantonCode        

In [93]:
acc_types = pd.DataFrame(acc_data.iloc[:, 2].unique(), columns=["acc_type"]).dropna()
cantons = pd.DataFrame(acc_data.iloc[:, 13].unique(), columns=["canton"]).dropna()

a = {"acc_type": "a"}
z = {"acc_type": "z"}

acc_types = acc_types.append([a, z], ignore_index=True)
cantons

,canton
0,ZH
1,FR
2,GE
3,NE
4,TG
5,BE
6,SO
7,BL
8,JU
9,VD


In [94]:
acc_data_index = cantons.merge(acc_types, how="cross")
len(acc_data_index)

338

In [103]:
acc_data_old = acc_data.iloc[:, [13, 2, 27]].groupby(["CantonCode", "AccidentType"]).sum()
acc_data_old.head()

acc_count
CantonCode AccidentType           
AG         at0                49.0
           at00                1.0
           at1                 8.0
           at2                 9.0
           at3                 6.0

In [104]:
acc_data_new = acc_data_index.merge(acc_data_old, how="left", left_on=["canton", "acc_type"], right_index=True)
acc_data_new.set_index(["canton", "acc_type"]).fillna(0)

acc_count
canton acc_type           
ZH     at0           143.0
       at4             8.0
       at8            68.0
       at6             3.0
       at00           10.0
...                    ...
AI     at2             1.0
       at9             0.0
       at7             0.0
       a               0.0
       z               0.0

[338 rows x 1 columns]

In [105]:
road_types = pd.DataFrame(acc_data.iloc[:, 9].unique(), columns=["road_type"]).dropna()
road_types

a1 = {"road_type": "a"}
z1 = {"road_type": "z"}

road_types = road_types.append([a1, z1], ignore_index=True)
road_types

,road_type
0,rt432
1,rt433
2,rt430
3,rt431
4,rt439
5,rt434
6,a
7,z


In [109]:
road_data_index = cantons.merge(road_types, how="cross")
road_data_index.head()

,canton,road_type
0,ZH,rt432
1,ZH,rt433
2,ZH,rt430
3,ZH,rt431
4,ZH,rt439


In [107]:
acc_data_old_road = acc_data.iloc[:, [13, 9, 27]].groupby(["CantonCode", "RoadType"]).sum()
acc_data_old_road.head()

acc_count
CantonCode RoadType           
AG         rt430          18.0
           rt431           1.0
           rt432          73.0
           rt433          57.0
           rt439           4.0

In [111]:
road_data_new = road_data_index.merge(acc_data_old_road, how="left", left_on=["canton", "road_type"], right_index=True)
road_data_new = road_data_new.set_index(["canton", "road_type"], drop=True).fillna(0)
road_data_new.head()

acc_count
canton road_type           
ZH     rt432           88.0
       rt433          156.0
       rt430           21.0
       rt431            5.0
       rt439           12.0

In [112]:
road_data_new.to_csv("tableau_road_types.csv")